In [1]:
!pip install imblearn
!pip install xgboost
!pip install --user numba shap

In [2]:
!pip install --upgrade scikit-learn --user

# Projet de Modèle de Scoring

## Introduction
Ce projet vise à développer un modèle de scoring de crédit qui prédit la probabilité qu'un client rembourse son crédit. Ce modèle aidera la société financière à prendre des décisions éclairées sur l'octroi de prêts.


## Importation des Libraries

In [3]:
import sklearn
print(sklearn.__version__)

1.5.0


In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, FixedThresholdClassifier
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_imblearn
from imblearn.under_sampling import RandomUnderSampler
import mlflow
import mlflow.sklearn
from collections import Counter
from xgboost import XGBClassifier

# Configuration de MLFlow

In [5]:
import mlflow
from mlflow import log_metric, log_param, log_artifacts

# Définir l'URI de suivi de MLFlow
path_mlruns = '../mlruns/'
mlflow.set_tracking_uri(path_mlruns)

# Affiche le URI de tracking actuel
print(mlflow.get_tracking_uri())

../mlruns/


In [6]:
'''
# Créer une nouvelle expérience et obtenir son ID
experiment_id = mlflow.create_experiment("model_scoring")
print(f"ID de la nouvelle expérience : {experiment_id}")
'''

'\n# Créer une nouvelle expérience et obtenir son ID\nexperiment_id = mlflow.create_experiment("model_scoring")\nprint(f"ID de la nouvelle expérience : {experiment_id}")\n'

## Chargement des Données

In [7]:
path_data_processed = "../data/processed/"
data = pd.read_csv(path_data_processed + 'processed_data_train.csv', index_col=[0])
data_test = pd.read_csv(path_data_processed + 'processed_data_test.csv', index_col=[0])

## Séparation des données en ensembles d'entraînement et de test

In [8]:
X = data.drop('TARGET', axis=1)
y = data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Définition des Fonctions Utiles

In [9]:
def plot_roc_curve(y_test, y_scores, title="ROC Curve"):
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    auc = roc_auc_score(y_test, y_scores)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend()
    plt.show()
    return auc

In [10]:
def business_cost(y_true, y_pred, cost_fn=10, cost_fp=1):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return -(cost_fn * fn + cost_fp * fp)  # Minimize business cost

In [11]:
def business_score(y_true, y_pred, cost_fn=cost_fn, cost_fp=cost_fp):
    """
    Calculate the business score for predictions.

    Parameters:
    - y_true: array, true labels.
    - y_pred: array, predicted labels.
    - cost_fn: int, cost associated with a false negative.
    - cost_fp: int, cost associated with a false, positive.

    Returns:
    - float, normalized business score between 0 and 1.
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    total_cost = cost_fn * fn + cost_fp * fp
    max_cost = cost_fn * (fn + tp) + cost_fp * (fp + tn)  # worst case scenario: all predictions are wrong
    
    # Normalize and subtract from 1 to flip the scale: higher is better
    return 1 - (total_cost / max_cost)

## Prétraitement des Données
### Vérification des Valeurs Manquantes

In [12]:
print(f"Missing values: {data.isna().sum().sum()} ({data.isna().sum().sum()/data.size:.2%})")

Missing values: 36088183 (19.62%)


In [13]:
# Affichage des colonnes pour vérification
print(data.columns)

# Définition des features numériques et catégorielles
numeric_features = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
categorical_features = [col for col in X.columns if X[col].dtype == 'object']

# Affichage des types de features
print("Numeric Features:", numeric_features)
print("Categorical Features:", categorical_features)

Index(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'REGION_POPULATION_RELATIVE',
       ...
       'PREV_PRODUCT_COMBINATION_Cash X-Sell: low_MEAN',
       'PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN',
       'PREV_PRODUCT_COMBINATION_POS household with interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS household without interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS other with interest_MEAN',
       'PREV_PRODUCT_COMBINATION_POS others without interest_MEAN'],
      dtype='object', length=598)
Numeric Features: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CH

### Création de pipelines et de transformations

In [14]:
# Définition des transformations pour les données numériques et catégorielles
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Entraînement et évaluation de XGBoost

In [ ]:
# Démarrage de la session MLFlow
mlflow.start_run(run_name="Optimized XGBoost with Fixed Thresholds", experiment_id=313886723643097310)

# Configuration de MLFlow pour le suivi des métriques
def log_metrics(y_true, y_pred, y_proba, threshold):
    auc = roc_auc_score(y_true, y_proba[:, 1])
    business_score_value = business_score(y_true, (y_proba[:, 1] > threshold).astype(int))
    mlflow.log_metric("AUC", auc)
    mlflow.log_metric("best_business_score", business_score_value)
    mlflow.log_metric("Optimized Threshold", threshold)

# Définition de la pipeline avec XGBoost et FixedThresholdClassifier
model_xgb = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=10)
fixed_threshold_classifier = FixedThresholdClassifier(estimator=model_xgb, threshold=0.5, response_method='predict_proba')

model_xgb_pipeline = make_pipeline_imblearn(
    preprocessor,
    SMOTE(sampling_strategy=0.5, random_state=42),
    fixed_threshold_classifier
)

# Paramètres pour GridSearchCV
param_grid = {
    'fixedthresholdclassifier__estimator__max_depth': [3, 5],
    'fixedthresholdclassifier__estimator__learning_rate': [0.1, 0.2],
    'fixedthresholdclassifier__estimator__n_estimators': [100, 200],
    'fixedthresholdclassifier__estimator__subsample': [0.7, 1],
    'fixedthresholdclassifier__threshold': [0.1, 0.5, 1.0]
}

# Configuration de GridSearchCV avec une métrique personnalisée
business_scorer = make_scorer(business_score, greater_is_better=True, needs_proba=False, cost_fn=10, cost_fp=1)
grid_search = GridSearchCV(model_xgb_pipeline, param_grid, scoring=business_scorer, cv=3, verbose=2)
grid_search.fit(X_train, y_train)

# Récupération des résultats
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)

# Log des métriques
log_metrics(y_test, y_pred, y_proba, best_params['fixedthresholdclassifier__threshold'])
print("Meilleurs paramètres : ", best_params)
print("Meilleur score business : ", best_score)

# Log dans MLFlow
mlflow.log_params(best_params)
mlflow.log_metric("best_business_score", best_score)

# Vérification et log des importances des caractéristiques
if hasattr(best_model.named_steps['fixedthresholdclassifier'].estimator, 'feature_importances_'):
    feature_importances = best_model.named_steps['fixedthresholdclassifier'].estimator.feature_importances_
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df.sort_values('Importance', ascending=False, inplace=True)
    print(feature_importance_df.head(20))

mlflow.sklearn.log_model(best_model, "model_v1")
mlflow.end_run()

In [ ]:
bbbb

In [15]:
mlflow.start_run(run_name="Optimized XGBoost with Fixed Thresholds", experiment_id=313886723643097310)

# Pipeline de modélisation avec XGBoost et FixedThresholdClassifier
model_xgb = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=10)
fixed_threshold_classifier = FixedThresholdClassifier(estimator=model_xgb, threshold=0.5, response_method='predict_proba')

model_xgb_pipeline = make_pipeline_imblearn(
    preprocessor,
    SMOTE(sampling_strategy=0.5, random_state=42),
    fixed_threshold_classifier
)

# Paramètres pour GridSearchCV
param_grid = {
    'fixedthresholdclassifier__estimator__max_depth': [3, 5],
    'fixedthresholdclassifier__estimator__learning_rate': [0.1, 0.2],
    'fixedthresholdclassifier__estimator__n_estimators': [100, 200],
    'fixedthresholdclassifier__estimator__subsample': [0.7, 1],
    'fixedthresholdclassifier__threshold': [0.1, 0.3, 0.5, 0.8, 1.0]
}

# Configuration de GridSearchCV
#scorer = make_scorer(threshold_auc_score, needs_proba=True)
business_scorer = make_scorer(business_score, greater_is_better=True, needs_proba=False, cost_fn=10, cost_fp=1)
grid_search = GridSearchCV(model_xgb_pipeline, param_grid, scoring=business_scorer, cv=3, verbose=2)
grid_search.fit(X_train, y_train)

# Afficher les résultats
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur score business : ", grid_search.best_score_)

# Configuration de MLFlow pour le suivi
mlflow.log_params(grid_search.best_params_)
mlflow.log_metric("best_business_score", grid_search.best_score_)
mlflow.sklearn.log_model(grid_search.best_estimator_, "model_v1")
mlflow.end_run()


C:\Users\Farid\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Fitting 3 folds for each of 80 candidates, totalling 240 fits


C:\Users\Farid\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Farid\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time=  56.9s
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time=  52.4s
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time=  56.2s
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_

[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.3min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.3min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.3min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.1, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_

...

[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.2min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.2min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.3min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=3, fixedthresholdclassifier__estimator__n_

[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.4min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.4min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=100, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.4min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_

[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.7min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.9min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_estimators=200, fixedthresholdclassifier__estimator__subsample=0.7, fixedthresholdclassifier__threshold=0.1; total time= 1.9min
[CV] END fixedthresholdclassifier__estimator__learning_rate=0.2, fixedthresholdclassifier__estimator__max_depth=5, fixedthresholdclassifier__estimator__n_

Meilleurs paramètres :  {'fixedthresholdclassifier__estimator__learning_rate': 0.1, 'fixedthresholdclassifier__estimator__max_depth': 5, 'fixedthresholdclassifier__estimator__n_estimators': 200, 'fixedthresholdclassifier__estimator__subsample': 1, 'fixedthresholdclassifier__threshold': 0.1}
Meilleur score AUC :  0.7046460240848514


C:\Users\Farid\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Importance des caractéristiques
Extraction de l'importance des caractéristiques après l'entraînement des modèles :
* **Ajustement du préprocesseur :** J'ai ajusté le preprocessor sur les données d'entraînement, ce qui est essentiel pour préparer les features avant de récupérer leur nom.

* **Gestion des features catégorielles :** J'ai conditionné l'ajustement des caractéristiques catégorielles et la récupération de leurs noms. C'est utile si la liste des caractéristiques catégorielles peut varier ou être vide.

* **Extraction et affichage des importances des caractéristiques :** Après avoir entrainé le modèle, j'ai extrait l'importance de chaque caractéristique et affiché les plus importantes. Cela nous aide à identifier quelles variables ont le plus d'impact sur les prédictions du modèle.

In [17]:
# Assurez-vous que le préprocesseur est ajusté
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['NAME_CONTRACT_TYPE', 'CODE_GENDER',
                                  'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
                                  'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
                                  'AMT_CREDIT', 'AMT_ANNUITY',
                                  'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
                                  'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
                                  'DAYS_ID_PUB...
                                  'REGION_RATING_CLIENT_W_CITY',
                                  'HOUR_APPR_PROCESS_START',
                                  'REG_REGION_NOT_LIVE_REGION',
                                  'REG_REGION_NOT_WORK_REGION',
                                  'LIVE_REGION_NOT_WORK_REGION',
                                  'REG_CITY_NOT_LIVE_CITY',
                                  'REG_CITY_NOT_WORK_CITY',
                                  'LIVE_CITY_NOT_WORK_CITY', ...]),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [])])

In [18]:
# Récupération des noms des caractéristiques
feature_names = list(preprocessor.transformers_[0][1].get_feature_names_out())
if categorical_features:
    feature_names += list(preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features))

In [19]:
# Récupération des importances des caractéristiques du modèle entraîné
feature_importances = grid_search.best_estimator_.named_steps['fixedthresholdclassifier'].estimator.feature_importances_

# Création et affichage du DataFrame
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df.sort_values('Importance', ascending=False, inplace=True)
print(feature_importance_df.head(20))

NotFittedError: need to call fit or load_model beforehand

In [20]:
# Vérifiez si le grid search a trouvé un meilleur estimateur
if hasattr(grid_search.best_estimator_.named_steps['fixedthresholdclassifier'].estimator, 'feature_importances_'):
    feature_importances = grid_search.best_estimator_.named_steps['fixedthresholdclassifier'].estimator.feature_importances_

    # Création et affichage du DataFrame
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df.sort_values('Importance', ascending=False, inplace=True)
    print(feature_importance_df.head(20))
else:
    print("Le modèle n'est pas correctement ajusté.")


Le modèle n'est pas correctement ajusté.


In [ ]:
# Ajustement du préprocesseur sur les données d'entraînement
preprocessor.fit(X_train)

In [ ]:
# S'assurer que preprocessor a des features catégorielles à traiter
if categorical_features:
    preprocessor.fit(X_train[categorical_features])
    feature_names_cat = list(preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features))
    feature_names = list(preprocessor.transformers_[0][1].get_feature_names_out()) + feature_names_cat
else:
    feature_names = list(preprocessor.transformers_[0][1].get_feature_names_out())

In [ ]:
# Récupération des importances des caractéristiques du modèle entraîné
feature_importances = grid_search.best_estimator_.named_steps['xgbclassifier'].feature_importances_

# Affichage des importances des caractéristiques
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
print(feature_importance_df.head(20))

## Outils d'interprétation des modèles
Utilisation de SHAP pour l'interprétation des prédictions du modèle XGBoost :

In [ ]:
import shap

# Calcul des valeurs SHAP
explainer = shap.TreeExplainer(grid_search.best_estimator_.named_steps['xgbclassifier'])
shap_values = explainer.shap_values(X_train)

# Visualisation
shap.summary_plot(shap_values, X_train, feature_names=feature_names)

In [ ]:
# Création d'un objet Explanation pour les données de test
explainer = shap.Explainer(grid_search.best_estimator_.named_steps['xgbclassifier'])
shap_values_test = explainer(X_test)

# Visualisation pour une instance spécifique des données de test
index = 0  # Index de l'instance à expliquer
shap.force_plot(explainer.expected_value, shap_values_test[index].values, X_test.iloc[index])

# Visualisation de l'importance globale des caractéristiques
shap.summary_plot(shap_values_test.values, X_test, plot_type="bar")

# Diagramme détaillé pour une instance spécifique
shap.waterfall_plot(shap_values_test[index])